In [1]:
from transformers import pipeline

model_id = "cjvt/GaMS-2B-Instruct"

pline = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto" 
)


d:\5_programi\590_programming\Miniconda3\envs\nlp-project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]
Device set to use cpu


In [2]:
import jinja2
# Example of response generation
message = [{"role": "user", "content": "Kateri je najpomembnejši dogodek v slovenski zgodovini?"}]
response = pline(message, max_new_tokens=512)
print("Model's response:", response[0]["generated_text"][-1]["content"])


Model's response: Najpomembnejši dogodek v slovenski zgodovini je bil razpad Avstro-Ogrske in priključitev Prekmurja k Sloveniji.



In [9]:

# Example of conversation chain
new_message = response[0]["generated_text"]
new_message.append({"role": "user", "content": "Lahko bolj podrobno opišeš ta dogodek?"})
new_message = new_message[:3]


In [10]:


response = pline(new_message, max_new_tokens=300)
print("Model's response:", response[0]["generated_text"][-1]["content"])


Model's response: Razpad Avstro-Ogrske je bil posledica prve svetovne vojne in razpada Avstro-Ogrskega imperija. Po vojni je bila ustanovljena Kraljevina SHS, ki je vključevala večino slovenskega etničnega ozemlja, vključno s Prekmurjem. Prekmurje je bilo dolgo časa pod madžarsko oblastjo, zato je bilo to priključitev še posebej pomembno za ohranjanje slovenskega jezika in kulture.



In [ ]:
prompt = """Classify the text into neutral, negative or positive.
Here are some annotated examples:
Example 1:
Text: 'This restaurant is the best I've ever been to. The food was delicious and the staff were very friendly.'
Sentiment: positive

Example 2:
Text: 'I was disappointed with my purchase. The product broke within a week.'
Sentiment: negative

Example 3:
Text: 'The movie was okay, not great but not bad either.'
Sentiment: neutral

Text: 'I absolutely love the new Spider-Man movie. It's incredibly well done!'
Sentiment:"""
res = pipeline(prompt)
print(res[0]["generated_text"])